In [1]:
import pandas as pd

In [5]:
# 编码DRUG和TAR
df_d = pd.read_csv("../data/raw/Feature_DRUG.csv")
df_t = pd.read_csv("../data/raw/Feature_TAR.csv")

drug_ids = df_d['DrugID']
tar_ids = df_t['TargetID']

drug_vocab = []
for id in drug_ids:
    drug_vocab.append("Drug|||" + str(id) + "\n")

tar_vocab=[]
for id in tar_ids:
    tar_vocab.append("Target|||" + str(id) + "\n")

with open("../data/processed/drug_vocab.txt", 'w') as file:
    file.writelines(drug_vocab)

with open("../data/processed/tar_vocab.txt",'w') as file:
    file.writelines(tar_vocab)

In [6]:
with open("../data/processed/drug_vocab.txt", 'r') as file:
    drug_vocab = file.readlines()

with open("../data/processed/tar_vocab.txt", 'r') as file:
    tar_vocab = file.readlines()

drug_id_dict = {}
for i in range(len(drug_vocab)):
    drug_id_dict[drug_vocab[i][:-1]] = i

tar_id_dict = {}
for i in range(len(tar_vocab)):
    tar_id_dict[tar_vocab[i][:-1]] = i


def get_id(ID, is_drug=True):
    if is_drug:
        key = "Drug|||" + str(ID)
        return drug_id_dict[key]
    else:
        key = "Target|||" + str(ID)
        return tar_id_dict[key]


# drug_drug edge_index 和 edge_att
edge_index = []
edge_att = []
e_dd = pd.read_csv("../data/raw/Edge_DRUG-DRUG.csv", dtype='str')

cols = e_dd.columns[1:]
for index, row in e_dd.iterrows():
    id = get_id(row['DrugID'])
    for col in cols:
        if str(row['DrugID']) == str(col):
            continue
        edge_index.append([id, get_id(col)])
        edge_att.append(e_dd.loc[index, col])
with open("../data/processed/drug_drug_edge_index.txt", 'w') as file:
    for e_i in edge_index:
        file.write(str(e_i))
        file.write("\n")

with open("../data/processed/drug_drug_edge_att.txt", 'w') as file:
    for e_t in edge_att:
        file.write(str(e_t))
        file.write("\n")

In [7]:

# target_target edge_index 和 edge_att
edge_index = []
edge_att = []
e_dd = pd.read_csv("../data/raw/Edge_TAR-TAR.csv", dtype='str')

for index, row in e_dd.iterrows():
    id1 = get_id(row['TargetID1'], False)
    id2 = get_id(row['TargetID2'], False)

    edge_index.append([id1,id2])
    edge_att.append(1.0)

with open("../data/processed/tar_tar_edge_index.txt", 'w') as file:
    for e_i in edge_index:
        file.write(str(e_i))
        file.write("\n")

with open("../data/processed/tar_tar_edge_att.txt", 'w') as file:
    for e_t in edge_att:
        file.write(str(e_t))
        file.write("\n")

In [8]:

# drug_target edge_index 和 edge_att
edge_index = []
edge_att = []
e_dd = pd.read_csv("../data/raw/Edge_DRUG-TAR.csv", dtype='str')

for index, row in e_dd.iterrows():
    id1 = get_id(row['DrugID'])
    id2 = get_id(row['TargetID'], False)

    edge_index.append([id1,id2])
    edge_att.append(1.0)

with open("../data/processed/drug_tar_edge_index.txt", 'w') as file:
    for e_i in edge_index:
        file.write(str(e_i))
        file.write("\n")

with open("../data/processed/drug_tar_edge_att.txt", 'w') as file:
    for e_t in edge_att:
        file.write(str(e_t))
        file.write("\n")

In [2]:
# 数据集打乱
df=pd.read_csv("../data/raw/Label_DRUGCOMB.csv")
from sklearn.utils import shuffle
df = shuffle(df)
df.to_csv("../data/processed/Shuffled_Label_Data.csv")

In [7]:
# results.txt 转 csv
txt_file="../data/result/GATNet(DrugA_DrugB)0--AUCs.txt"
with open(txt_file,'r') as f:
    texts=f.readlines()

res=[]
for text in texts:
    res.append(text[:-1].split('\t'))

df=pd.DataFrame(data=res)
df.to_csv("../data/result/GATNet_results.csv",header=0,index=0)